In [75]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


In [76]:
data= sc.textFile('Downloads/hollins.dat.gz').map(lambda x:(x.split(" ")[0], x.split(" ")[1]))

In [77]:
links = data.filter(lambda keyValue: "http://" not in keyValue[1]) 

In [78]:
ranks_str = data.filter(lambda keyValue: "http://" in keyValue[1]) 
ranks = ranks_str.mapValues(lambda value:1)
ranks.take(10)

[('1', 1),
 ('2', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1),
 ('6', 1),
 ('7', 1),
 ('8', 1),
 ('9', 1),
 ('10', 1)]

In [79]:
ranks.count()

6012

In [80]:
links_float = links.mapValues(lambda value:int(value))

In [81]:
links_noheader= links.filter(lambda keyValue:keyValue[1]!='23875')
links_noheader.count()
links_noheader.filter(lambda keyValue:keyValue[0]=='1').reduceByKey(lambda x,y:x+' '+y).mapValues(lambda x:list(x.split(" "))).take(1)


[('1',
  ['2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '24',
   '25'])]

In [82]:
links_agg=links_noheader.reduceByKey(lambda x,y:x+' '+y).mapValues(lambda x:list(x.split(" "))).partitionBy(4).persist()
print(links_agg.take(3))
links_agg.count()

[('145', ['2', '27', '28', '31', '37', '38', '43', '52', '61', '73', '138', '164', '614', '625', '626']), ('499', ['1520', '1521', '1522', '1523', '1524', '1525', '1526', '1527', '1528', '1529', '1530', '1531', '1532']), ('3188', ['3789', '3790', '3791'])]


2823

In [83]:
links_agg.join(ranks).count()

2823

In [84]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)
for iteration in range(10):
    # Calculates URL contributions to the rank of other URLs.
    contributions = links_agg.join(ranks).flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
    print(contributions.take(1)) 
     # Re-calculates URL ranks based on neighbor contributions.
    ranks = contributions.reduceByKey(lambda x,y:x+y).mapValues(lambda rank: rank * 0.85 + 0.15)
    print(contributions.reduceByKey(lambda x,y:x+y).take(1))
# Collects all URL ranks and dump them to console.
for (link, rank) in ranks.take(3):
    print("%s has rank: %s." ,(link, rank))

[('425', 1.0)]
[('131', 0.043478260869565216)]
[('822', 0.027142857142857142)]
[('1805', 0.010997442455242964)]
[('2', 0.15459133356705926)]
[('18', 1.372363884993737)]
[('2', 0.017513451780923447)]
[('5888', 0.017684670781893)]
[('2', 0.033162463093445185)]
[('1805', 0.014936452136012177)]
[('2', 0.012375241937305449)]
[('1344', 0.012408791636322258)]
[('2', 0.013937331001644375)]
[('1469', 0.050306803047224166)]
[('3907', 0.18755438762687418)]
[('3222', 2.243810638105895)]
[('2', 0.030631263544996096)]
[('289', 0.17997936654849866)]
[('3189', 0.4853992545846504)]
[('283', 0.17676268645623472)]
%s has rank: %s. ('283', 0.3002482834877995)
%s has rank: %s. ('2707', 0.1535285493858371)
%s has rank: %s. ('5494', 0.20868193308173105)


In [85]:
ranks.takeOrdered(3,lambda atuple: -atuple[1])

[('2', 46.10691105759518),
 ('37', 24.468079296370252),
 ('38', 22.67016158801036)]

In [95]:
links_noheader.filter(lambda keyValue:keyValue[0]=='2').reduceByKey(lambda x,y:x+' '+y).mapValues(lambda x:list(x.split(" "))).take(1)

[('2',
  ['26',
   '27',
   '28',
   '29',
   '30',
   '31',
   '32',
   '33',
   '34',
   '35',
   '36',
   '37',
   '38',
   '39',
   '40',
   '41',
   '42',
   '43',
   '44',
   '45',
   '46',
   '47',
   '48',
   '49',
   '50'])]

In [87]:
links_noheader.filter(lambda keyValue:keyValue[0]=='6').take(20)

[('6', '190'), ('6', '191')]

In [ ]:
ranks.saveAsTextFile('ranks_new3')

In [46]:
links_agg.join(ranks).flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1])).take(10)

[('2', 0.012331823386921743),
 ('34', 0.012331823386921743),
 ('37', 0.012331823386921743),
 ('38', 0.012331823386921743),
 ('43', 0.012331823386921743),
 ('52', 0.012331823386921743),
 ('73', 0.012331823386921743),
 ('148', 0.012331823386921743),
 ('174', 0.012331823386921743),
 ('175', 0.012331823386921743)]

In [94]:
print(ranks.filter(lambda keyValue:keyValue[0]=='2').collect())

[('2', 46.10691105759518)]
